In [1]:
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import csv
import numpy as np

sia =SentimentIntensityAnalyzer()
def get_sentiment_of(sentence_in):
    try:
        scrs=sia.polarity_scores(sentence_in)
        the_important_one=scrs['compound']
    except:
        the_important_one=0
    return the_important_one

df = pd.read_csv('yelp_reviews_chicago_labeled.csv')
df2 = df.drop(['reviewID', 'restaurantID', 'reviewerID', 'date', 'name', 'location', 'yelpJoinDate', 'mnr', 'rl', 'rd', 'Maximum Content Similarity'], axis = 1)
df2 = df2.drop(['friendCount', 'reviewCount', 'firstCount', 'usefulCount', 'complimentCount', 'tipCount', 'fanCount', 'restaurantRating'], axis = 1)
f = df2['reviewContent']
sent = list()
for rows in f:
    #print(rows)
    sent.append(get_sentiment_of(rows))
print(len(sent))

df2['Sentiment'] = sent
df_final = df2.drop(['reviewContent'], axis = 1)
fields_with_nan_length = df_final.isna().sum().where(lambda x : x > 0).dropna()
df_final = df_final.iloc[:-1, :]
authentic_reviews_df = df_final[df_final['flagged'] == 'N']
fake_reviews_df = df_final[df_final['flagged'] == 'Y']
authentic_reviews_df = authentic_reviews_df.sample(n = 500)
df_final = pd.concat([authentic_reviews_df , fake_reviews_df])
df_final['flagged'] = [0 if x == 'N' else 1 for x in df_final['flagged']]
flagged = df_final['flagged']
df_final = df_final.drop(['flagged'], axis=1)
df_final = pd.concat([df_final,flagged], axis=1)
X = df_final.iloc[:, :-1]
y = flagged
X.head()



3626


,rating,reviewUsefulCount,coolCount,funnyCount,Sentiment
1983,5,0,37.0,46.0,0.9796
667,5,0,26.0,17.0,0.9913
1491,4,1,92.0,60.0,0.9945
2546,4,0,494.0,323.0,0.8271
648,5,0,4.0,32.0,0.6369


In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [4]:
clf = DecisionTreeClassifier(random_state=0)
scores = cross_val_score(clf, X, y, cv = 10)
print(scores)

from sklearn.model_selection import cross_validate

print("Overal Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.84693878 0.82653061 0.80612245 0.7755102  0.82474227 0.82474227
 0.69072165 0.71134021 0.84536082 0.73195876]
Overal Accuracy: 0.79 (+/- 0.11)


In [5]:
train_copy = df_final.drop(['flagged'], axis=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
feat_labels = train_copy.columns[0:]
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("%2d %-*s %f" %(f+1,30,feat_labels[indices[f]],importances[indices[f]]))

 1 funnyCount                     0.476346
 2 Sentiment                      0.268856
 3 reviewUsefulCount              0.126381
 4 coolCount                      0.074709
 5 rating                         0.053707


In [6]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.79      0.84      0.81       146
     class 1       0.83      0.78      0.80       147

   micro avg       0.81      0.81      0.81       293
   macro avg       0.81      0.81      0.81       293
weighted avg       0.81      0.81      0.81       293

